1. if user read < 5 books call content filtering py
2.  if user read > 5 books call collaborative ratings py
3. if user read > 5 and review > 5  books call collaborative ratings and reviews .py

for 1. get 5th rec book and re input into content filtering and get top rec and raplce in oriignal recs
same idea for 2 and 3

In [1]:
import pandas as pd
import pickle
import faiss
import numpy as np
import joblib
from torch_geometric.nn import GATv2Conv
import torch

In [2]:
books_list = []

with open('../Pickle/books.pkl', 'rb') as file:
    while True:
        try:
            chunk = pickle.load(file)
            books_list.append(chunk)
        except EOFError:
            break  # Stop when end of file is reached
books = pd.concat(books_list, ignore_index=True)
books = books.drop_duplicates(subset='title', keep='first')

In [3]:
interactions = pd.read_pickle('../Pickle/interactions.pkl')
read = pd.read_pickle('../Pickle/read.pkl')

In [4]:
valid_book_ids = set(books['book_id'])
interactions = interactions[interactions['book_id'].isin(valid_book_ids)]
read = read[read['book_id'].isin(valid_book_ids)]

In [7]:
with open('../Pickle/umap_embeddings.pkl', 'rb') as f:
    umap_embeddings = pickle.load(f)

faiss_index = faiss.read_index('../Pickle/faiss_index.bin')

with open('../Pickle/book_id_to_index.pkl', 'rb') as f:
    book_id_to_index = pickle.load(f)

model = torch.load('../Pickle/gat_model.pth')

In [8]:
user_id = 123

In [9]:
user_books_read = read[read['user_id'] == user_id ]
user_num_books = user_books_read['book_id'].nunique()
user_reviews = interactions[interactions['user_id']== user_id]
user_num_reviews = user_reviews['book_id'].nunique()

In [14]:
user_num_reviews

8

In [10]:
def get_recommendations_by_cluster(book_id, books=books, umap_embeddings=umap_embeddings, top_n=5, book_id_to_index=book_id_to_index):
    if book_id not in book_id_to_index:
        print(f"Warning: {book_id} is not in book_id_to_index.")  # Print missing book_id
        return ("not in index")  # Return empty if book_id is not in book_id_to_index

    book_idx = book_id_to_index[book_id]
    input_book_title = books.loc[books['book_id'] == book_id, 'title'].values[0]

    # Search for nearest neighbors using FAISS
    distances, indices = faiss_index.search(np.array([umap_embeddings[book_idx]]), top_n + 1)

    recommendations = []
    for idx, dist in zip(indices[0][1:], distances[0][1:]):  # Exclude the book itself
        # Check if the index is within bounds
        if idx >= len(books):
            print(f"Warning: Index {idx} is out of bounds for books DataFrame.")
            continue  # Skip if the index is out of bounds

        recommended_book = books.iloc[idx]
        explanation = f"Similarity Score: {round(1 / (1 + dist), 3)}"
        recommendations.append({
            "title": recommended_book["title"],
            "authors": recommended_book["authors"],
            "cluster": recommended_book.get("cluster", "N/A"),
            "explanation": explanation
        })

    return recommendations


In [11]:
def recommend_books(model, interactions, user_id, books_read, books, n_recommendations=5):
    """Recommend top-N books for a specific user or all users using the trained NMF model."""
    
    # Get unique books from the interactions dataset
    all_books = interactions['book_id'].unique()
    unread_books = list(set(all_books) - set(books_read))

    # Predict ratings for unread books
    user_predictions = [
        (book_id, model.predict(uid=user_id, iid=book_id).est) for book_id in unread_books
    ]
    
    # Sort predictions by estimated rating in descending order
    user_predictions.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top-N recommended books and map to title, authors, and rating
    top_books = user_predictions[:n_recommendations]
    top_books_with_details = []
    
    for book_id, rating in top_books:
        # Lookup the title and authors of the book using the books DataFrame
        book_info = books.loc[books['book_id'] == book_id, ['title', 'authors']].values[0]
        book_title = book_info[0]
        book_authors = book_info[1]
        
        # Append the book_id, title, authors, and rating to the list
        top_books_with_details.append((book_id, book_title, book_authors, rating))
        
    return {user_id: top_books_with_details}  # Return list of (book_id, title, authors, predicted_rating)


In [13]:
if user_num_books < 5:
    print(f"User {user_id} has read {user_num_books} books. Using content-based filtering.")
    recommendations = []

# For each book the user has read, get recommendations
    for book_id in user_books_read:
        book_recommendations = get_recommendations_by_cluster(book_id)
        recommendations.extend(book_recommendations)

# Remove duplicate recommendations
    unique_recommendations = {rec['title']: rec for rec in recommendations}.values()

# Convert to a list for easier handling
    unique_recommendations = list(unique_recommendations)

# Display the recommendations
    for rec in unique_recommendations:
        print(f"Title: {rec['title']}")
        print(f"Authors: {rec['authors']}")
        print(f"Cluster: {rec['cluster']}")
        print(f"Explanation: {rec['explanation']}")
        print() 
        
if user_num_books > 5 and user_num_reviews < 5:

    print(f"User {user_id} has read {user_num_books} books. Using collaborative filtering.")
    best_nmf = joblib.load('../Pickle/best_nmf_model.pkl')
    book_recommendations = recommend_books(best_nmf, interactions, user_id, user_books_read , books)
    user_predictions = book_recommendations.get(user_id, [])
    print(user_predictions)
    fifth_rec = user_predictions[3][0]  # book_id of the 5th recommendation
     # Feed it into the get_recommendation_by_cluster function to get 5 new books
    new_recommendations = get_recommendations_by_cluster(fifth_rec)
        # Take the 1st book from the new recommendations
    new_first_rec = new_recommendations[0]
        
        # Replace the original 5th recommendation with the new first recommendation
    user_predictions[4] = (new_first_rec, user_predictions[4][1])  # Keep the original rating
        
        # Return the updated recommendations for the user
    book_recommendations[user_id] = user_predictions[:5]
    print(user_predictions)

if user_num_books > 5 and user_num_reviews > 5:
    print("using nmf and gat2vconv")
    print(f"User {user_id} has read {user_num_books} books. Using collaborative filtering.")
    best_nmf = joblib.load('../Pickle/best_nmf_model.pkl')
    book_recommendations = recommend_books(best_nmf, interactions, user_id, user_books_read , books)
    user_predictions = book_recommendations.get(user_id, [])
    print(user_predictions)
    

using nmf and gat2vconv
User 123 has read 22 books. Using collaborative filtering.
[(236, 'The Complete Nonsense of Edward Lear', ['Edward Lear'], 5), (461, 'The Inner Life of Martin Frost', ['Paul Auster', 'Glenn Thomas'], 5), (790, 'Rough Guide Directions: London', ['Rob Humphreys'], 5), (941, 'Love As A Foreign Language #5', ['J. Torres', 'Eric  Kim'], 5), (1234, 'Shadows and Wind: A View of Modern Vietnam', ['Robert Templer'], 5)]
